In [ ]:
!pip install tpot
!pip install pipelineprofiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 948.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11937 sha256=e43087d52261150814a93500fc16c3b0f9c0b9c63927bbb65441db12b812eedb
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.1/881.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.5 MB/s eta 0:00:00


#HeartFailure

資料集連結:<br>
Heart Failure Prediction<br>
https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data

##TPOT

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
import PipelineProfiler
from joblib import dump, load
from sklearn.metrics import mean_squared_error, accuracy_score, average_precision_score, classification_report
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

# 讀取資料集
data = pd.read_csv('/content/drive/MyDrive/heart_failure_clinical_records_dataset.csv')

# 檢查並移除缺失值的列
data.dropna(inplace=True)

# 平衡數據集
def balance_data_with_smote(data):
    X = data.drop('DEATH_EVENT', axis=1)
    y = data['DEATH_EVENT']

    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    balanced_data = pd.DataFrame(X_resampled, columns=X.columns)
    balanced_data['DEATH_EVENT'] = y_resampled

    return balanced_data

# 使用 SMOTE 方法平衡數據集
df = balance_data_with_smote(data)

# 统计平衡後的 0 和 1 的数量
counts = df['DEATH_EVENT'].value_counts()
# 打印结果
print(counts)

# # 編碼字串型態的資料
# label_encoder = LabelEncoder()
# for column in data.columns:
#     if data[column].dtype == 'object':  # 只處理字串型態的資料
#         data[column] = label_encoder.fit_transform(data[column])

# 切分特徵和目標變數
x = data.drop('DEATH_EVENT', axis=1)
y = data['DEATH_EVENT']

# 資料打亂
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

# 定义搜索
clf = TPOTClassifier(generations=50,population_size=20, cv=10, scoring='accuracy', verbosity=5, random_state=1)
clf.fit(x_train, y_train)

# 查看最佳模型的交叉驗證評估
evaluated_individuals = clf.evaluated_individuals_

# 输出最佳模型
clf.export('tpot_model.py')

predicted = clf.predict(x_train)
print('Train loss (MSE):', mean_squared_error(
    y_train, predicted).round(5))
print('Train accuracy:', accuracy_score(
    y_train, predicted).round(5))
print(classification_report(y_train, predicted))

# 匯出模型
dump(clf.fitted_pipeline_, 'model.joblib')
# 匯入模型
clf = load('model.joblib')

predicted = clf.predict(x_test)
print('Prection loss (MSE):', mean_squared_error(
    y_test, predicted).round(5))
print('Prection accuracy:', accuracy_score(
    y_test, predicted).round(5))
print(classification_report(y_test, predicted))

1    203
0    203
Name: DEATH_EVENT, dtype: int64
32 operators have been imported by TPOT.


Optimization Progress:   0%|          | 0/1020 [00:00<?, ?pipeline/s]

_pre_test decorator: _random_mutation_operator: num_test=0 Negative values in data passed to MultinomialNB (input X).
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only dual=False, got dual=True.
_pre_test decorator: _random_mutation_operator: num_test=0 Found array with 0 feature(s) (shape=(50, 0)) while a minimum of 1 is required by GaussianNB..
_pre_test decorator: _mate_operator: num_test=0 'str' object has no attribute 'arity'.
_pre_test decorator: _random_mutation_operator: num_test=0 Solver lbfgs supports only dual=False, got dual=True.
_pre_test decorator: _mate_operator: num_test=0 'str' object has no attribute 'arity'.
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
_pre_test decorator: _mate_operator: num_test=0 'str' obje

In [ ]:
# 顯示評估過的所有個體及其詳細資訊
evaluated_individuals_details = evaluated_individuals
print(evaluated_individuals_details)

# 初始化一個字典來儲存模型和它們的準確率
model_scores = {}

# 遍歷每個評估過的個體
for model_string, model_info in evaluated_individuals_details.items():
    # 提取每個模型的平均準確率
    mean_accuracy = model_info['internal_cv_score']
    # 將模型及其準確率儲存在字典中
    model_scores[model_string] = mean_accuracy

# 找出準確率最高的模型
best_model_string = max(model_scores, key=model_scores.get)
best_model_score = model_scores[best_model_string]

print("Best Model:", best_model_string)
print("Best Model Score:", best_model_score)

{'DecisionTreeClassifier(RBFSampler(input_matrix, RBFSampler__gamma=0.4), DecisionTreeClassifier__criterion=entropy, DecisionTreeClassifier__max_depth=10, DecisionTreeClassifier__min_samples_leaf=12, DecisionTreeClassifier__min_samples_split=7)': {'generation': 0, 'mutation_count': 0, 'crossover_count': 0, 'predecessor': ('ROOT',), 'operator_count': 2, 'internal_cv_score': 0.5692028985507246}, 'GaussianNB(Nystroem(input_matrix, Nystroem__gamma=0.75, Nystroem__kernel=additive_chi2, Nystroem__n_components=5))': {'generation': 0, 'mutation_count': 0, 'crossover_count': 0, 'predecessor': ('ROOT',), 'operator_count': 2, 'internal_cv_score': 0.4981884057971014}, 'GradientBoostingClassifier(input_matrix, GradientBoostingClassifier__learning_rate=0.5, GradientBoostingClassifier__max_depth=4, GradientBoostingClassifier__max_features=0.8, GradientBoostingClassifier__min_samples_leaf=18, GradientBoostingClassifier__min_samples_split=11, GradientBoostingClassifier__n_estimators=100, GradientBoosti